# Library Import

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import analyzer
import neural_network as nn
import convolutional_neural_network as cnn
import visualizer

2024-11-12 17:53:31.706161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 17:53:31.710049: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 17:53:31.714910: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 17:53:31.726529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731462811.746246 3659957 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731462811.75

KeyboardInterrupt: 

# Training Data Set Preparation

In [ ]:
resolution = [250,250]
# Resolution should be consistent throughout the file

In [ ]:

# Because the training dataset are individual, the list of each data just contain themselves.
data1 = analyzer.Data("MORE_DATA/db_Y_0027.okc", resolution) 
data2 = analyzer.Data("MORE_DATA/db_Y_0030.okc", resolution)


Trying to open the file at MORE_DATA/db_Y_0027.okc...
Obtained metadata of the file at MORE_DATA/db_Y_0027.okc.
This data is sliced.
Trying to open the file at MORE_DATA/db_Y_0027.okc...
Obtained metadata of the file at MORE_DATA/db_Y_0027.okc.
Obtain the time for this data.
Trying to open the file at MORE_DATA/db_Y_0027.okc...
Obtained metadata of the file at MORE_DATA/db_Y_0027.okc.


# Training The Model

In [ ]:
# Run this section if you want to use NN model

array1, headers1, non_nan_indices1, num_grids1 = nn.data_arranger(data1.df)
array2, headers2, non_nan_indices2, num_grids2 = nn.data_arranger(data2.df)

# The learning rate, batch size, and epochs are proven to be working.

nn_model = nn.model_create_compile(headers1, 0.05)

nn_model, loss_hist = nn.model_train(nn_model, array1, 1000, 10)
nn_model, loss_hist = nn.model_train(nn_model, array2, 1000, 10)

Normalizing gradient datas...
Rearranging data to tensor for model classification...
Obtained regularized tensor.
Normalizing gradient datas...
Rearranging data to tensor for model classification...
Obtained regularized tensor.
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1421
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1141
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1139
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1139
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1140
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0907
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

In [ ]:
# Run this section if you want to use CNN model

array1, headers1, indices1 = cnn.data_arranger(data1.df, resolution)
array2, headers2, indices2 = cnn.data_arranger(data2.df, resolution)

# The learning rate and epochs are proven to be working.

cnn_model = cnn.model_2D_create_compile(headers1, 0.05, resolution)

cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array1, 3)
cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array2, 3)

# Data Classification

Choose codes from these 3 codes below to run if you want to classify few individual data.

In [ ]:
data3 = analyzer.Data("MORE_DATA/db_Y_0049.okc", resolution)

In [ ]:
# Run this code if your model choice is NN
array3, headers3, non_nan_indices3, num_grids3 = nn.data_arranger(data3.df)
data3.df = nn.model_classification(nn_model, array3, non_nan_indices3, num_grids3, data3.df, False)

In [ ]:
# Run this code if your model choice is CNN
array3, headers3, indices3 = cnn.data_arranger(data3.df, resolution)
data3.df = cnn.model_2D_classification(cnn_model, array3, indices3, data3.df, False)

Choose codes from these 3 codes below to run if you want to classify a series of data.

In [ ]:
list_paths_classify = [f"MORE_DATA/db_Y_{i:04d}.okc" for i in range(99)]

Data_classify = [analyzer.Data(path, list_paths_classify, resolution) for path in list_paths_classify] 

In [ ]:
# Run this code if your model choice is NN
for data in Data_classify:
    array, headers, non_nan_indices, num_grids = nn.data_arranger(data.df)
    data.df = nn.model_classification(nn_model, array, non_nan_indices, num_grids, data.df, False)

In [ ]:
# Run this code if your model choice is CNN
for data in Data_classify:
    array, headers, indices = cnn.data_arranger(data.df, data.resolution)
    data.df = cnn.model_2D_classification(cnn_model, array, indices, data.df, False)

# Data Export

Choose codes from these 2 codes below to run if you want to classify few individual data.

In [ ]:
# Run this code to export the image
visualizer.plot_2D_df(data3.df, 'is_boundary', 'classification.png')

In [ ]:
# Run this code to export the csv file
data3.df.to_csv('classification.csv', index=False)

Choose codes from these 2 codes below to run if you want to classify a series of data.

In [ ]:
# Run this code to export the image
for i, data in enumerate(Data_classify):
    visualizer.plot_2D_df(data.df, 'is_boundary', f'classification_{i}.png')

In [ ]:
# Run this code to export the csv file
for i, data in enumerate(Data_classify):
    data.df.to_csv(f'classification_{i}.csv', index=False)

# Temporary Code Zone
---

### Merge CSV

In [ ]:
import os
import csv

# 指定文件夹路径
path = '3D_DATA/small_db_result'
output_file = '3D_DATA/small_db.csv'

# 获取所有csv文件的文件名
files = [file for file in os.listdir(path) if file.endswith('.csv')]

# 打开输出文件，准备写入合并后的内容
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = None  # 创建一个CSV写入器，但先不初始化
    
    # 遍历所有CSV文件
    for index, filename in enumerate(files):
        # 显示进度
        print(f"merging file: {index + 1}/{len(files)}", flush=True)
        
        with open(os.path.join(path, filename), 'r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            # 获取文件的第一行（标题）
            header = next(reader)
            
            # 如果是第一个文件，写入标题
            if writer is None:
                writer = csv.writer(outfile)
                writer.writerow(header)
            
            # 从第二行开始读取并写入
            for row in reader:
                writer.writerow(row)

print("All files merged successfully!")


### Plot Streamline 2D

In [3]:
from visualizer import plot_3D_to_2D_slice_streamline
plot_3D_to_2D_slice_streamline(input_file="3D_DATA/ra10e7_result/ra10e7_100.csv", output_file="streamlines.html", direction='y', seed_points_resolution=[20,20], max_time=0.2, cmap = 'viridis', axis_limits=[-0.5,0.5,0,1])

### Plot Streamline 3D

In [2]:
from visualizer import plot_3D_streamline
plot_3D_streamline(input_file="3D_DATA/small_db.csv", output_file="streamlines.html", seed_points_resolution=[2,2,2])

KeyError: 'Data named "velocity" does not exist in this dataset.'